# JSON
- `JSON (JavaScript Object Notation)` is a leightweight data format for data exchange. In Python we have the built-in `json` module for encoding and decoding JSON data.
- Importing it with syntax `import json` and we are ready to work with JSON data:

#### Advantages Of JSON
- JSON exists as a "sequence of bytes" which is very useful in the case we need to transmit (stream) data over a network.
- Compared to XML, JSON is much smaller, translating into faster data transfers, and better experiences.
- JSON is extremely human-friendly since it is textual, and simultaneously machine-friendly.


#### JSON Format

````JSON
{
    "firstName": "Jane",
    "lastName": "Doe",
    "hobbies": ["running", "swimming", "singing"],
    "age": 28,
    "children": [
        {
            "firstName": "Alex",
            "age": 5
        },
        {
            "firstName": "Bob",
            "age": 7
        }
    ]
}
````

- JSON also supports primitive types (strings, numbers, boolean), as well as nested arrays and objects. <br>

Simple Python objects are translated to JSON according to the following table conversion: 

| Python  | JSON |
| ------------- | ------------- |
| dict  | object  |
| list, tuple  | array  |
| str  | string  |
| int, long, float| number  |
| True  | true  |
| False  | false  |
| None  | null |

#### From Python to JSON (Serialization, Encode)
We convert Python objects into a JSON string with the `json.dumps()` method.

In [2]:
import json

person = {"name": "John", "age": 30, "city": "New York", "hasChildren": False, "titles": ["engineer", "programmer"]}

# Converting into JSON
person_json = json.dumps(person)

#Converting with different format
person_json2 = json.dumps(person, indent=4, separators=("; ", "= "), sort_keys=True)

# the result is a JSON string:
print(person_json) 
print(person_json2) 

{"name": "John", "age": 30, "city": "New York", "hasChildren": false, "titles": ["engineer", "programmer"]}
{
    "age"= 30; 
    "city"= "New York"; 
    "hasChildren"= false; 
    "name"= "John"; 
    "titles"= [
        "engineer"; 
        "programmer"
    ]
}


We can also convert Python objects into JSON objects and save them into a file with the `json.dump()` method.



In [3]:
import json

person = {"name": "John", "age": 30, "city": "New York", "hasChildren": False, "titles": ["engineer", "programmer"]}

with open('person.json', 'w') as f:
    json.dump(person, f) # you can also specify indent etc...

#### FROM JSON to Python (Deserialization, Decode)
We can convert a JSON string into a Python object with the `json.loads()` method. The result will be a Python dictionary.

In [4]:
import json
person_json = """
{
    "age": 30, 
    "city": "New York",
    "hasChildren": false, 
    "name": "John",
    "titles": [
        "engineer",
        "programmer"
    ]
}
"""
person = json.loads(person_json)
print(person)

{'age': 30, 'city': 'New York', 'hasChildren': False, 'name': 'John', 'titles': ['engineer', 'programmer']}


We can also load data from a file and convert it to a Python object with the json.load() method.

In [5]:
import json

with open('person.json', 'r') as f:
    person = json.load(f)
    print(person)

{'name': 'John', 'age': 30, 'city': 'New York', 'hasChildren': False, 'titles': ['engineer', 'programmer']}


## Working With Custom Objects

#### Encoding
Encoding a custom object with the default `JSONEncoder` will raise a TypeError.<br>
We can specify a custom encoding function that will store the class name and all object variables in a dictionary.<br>
Using this function for the default argument in the `json.dump()` method.

In [6]:
import json
def encode_complex(z):
    if isinstance(z, complex):
        # just the key of the class name is important, the value can be arbitrary.
        return {z.__class__.__name__: True, "real":z.real, "imag":z.imag}
    else:
        raise TypeError(f"Object of type '{z.__class__.__name__}' is not JSON serializable")

z = 5 + 9j
zJSON = json.dumps(z, default=encode_complex)
print(zJSON)

{"complex": true, "real": 5.0, "imag": 9.0}


We can also create a custom Encoder class, and overwrite the `default()` method. Use this for the cls argument in the json.dump() method, or use the encoder directly.

In [7]:
from json import JSONEncoder
class ComplexEncoder(JSONEncoder):
    
    def default(self, o):
        if isinstance(z, complex):
            return {z.__class__.__name__: True, "real":z.real, "imag":z.imag}
        # Let the base class default method handle other objects or raise a TypeError
        return JSONEncoder.default(self, o)
    
z = 5 + 9j
zJSON = json.dumps(z, cls=ComplexEncoder)
print(zJSON)
# or use encoder directly:
zJson = ComplexEncoder().encode(z)
print(zJSON)

{"complex": true, "real": 5.0, "imag": 9.0}
{"complex": true, "real": 5.0, "imag": 9.0}


#### Decoding
Decoding a custom object with the defaut `JSONDecoder` is possible, but it will be decoded into a dictionary.<br>
Writing a custom decode function that will take a dictionary as input, and creates our custom object if it can find the object class name in the dictionary.<br>
Using this function for the `object_hook` argument in the `json.load()` method.

In [8]:
# Possible but decoded as a dictionary
z = json.loads(zJSON)
print(type(z))
print(z)

def decode_complex(dct):
    if complex.__name__ in dct:
        return complex(dct["real"], dct["imag"])
    return dct

# Now the object is of type complex after decoding
z = json.loads(zJSON, object_hook=decode_complex)
print(type(z))
print(z)

<class 'dict'>
{'complex': True, 'real': 5.0, 'imag': 9.0}
<class 'complex'>
(5+9j)
